# 00 数据接入与事件分析准备

**研究背景**: Charlie Kirk政治暗杀事件社交媒体舆论分析  
**事件时间**: 2025-09-10 枪击发生  
**数据时间窗口**: 2025-09-11 23:55 至 2025-09-13 00:12 (事件后72小时)

**目标**:
1. 加载原始CSV数据并规范化类型
2. 合并作者元数据
3. **添加事件相关时间字段**（核心优化）
4. 写入Parquet缓存供后续分析

In [1]:
import sys
from pathlib import Path

# 将项目根目录添加到 Python 路径
project_root = Path('/workspace')
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    
print(f"✅ Python 路径已配置: {project_root}")

✅ Python 路径已配置: /workspace


## 步骤 1: 加载原始数据

In [2]:
from src import io, analysis, profiling
import polars as pl
from datetime import datetime, timezone

# 加载原始推文数据 (LazyFrame)
raw_lf = io.scan_raw_tweets()
print(f"📊 数据 schema:")
print(raw_lf.collect_schema())

📊 数据 schema:
Schema([('pseudo_id', Int64), ('text', String), ('retweetCount', Int64), ('replyCount', Int64), ('likeCount', Int64), ('quoteCount', Int64), ('viewCount', Int64), ('bookmarkCount', Int64), ('createdAt', String), ('lang', String), ('isReply', String), ('pseudo_conversationId', Int64), ('pseudo_inReplyToUsername', String), ('pseudo_author_userName', Int64), ('quoted_pseudo_id', String), ('author_isBlueVerified', String)])


## 步骤 2: 数据清洗与类型规范化

In [3]:
# 收集数据并规范化布尔列
df = raw_lf.collect()
print(f"✅ 数据加载完成: {df.height:,} 行, {df.width} 列")

# 规范化布尔列
bool_columns = ['isReply', 'author_isBlueVerified']
df_cleaned = analysis.normalize_boolean_columns(df, bool_columns)
print(f"✅ 布尔列规范化完成: {bool_columns}")

# 转换时间字段为datetime类型
df_cleaned = df_cleaned.with_columns(
    pl.col('createdAt').str.to_datetime('%Y-%m-%d %H:%M:%S%#z').alias('createdAt')
)
print(f"✅ 时间字段转换完成")

# 【修复】转换 pseudo_inReplyToUsername 从 String 到 Int64
# 空字符串转换为 null，然后转为 Int64
df_cleaned = df_cleaned.with_columns(
    pl.when(pl.col('pseudo_inReplyToUsername') == '')
      .then(None)
      .otherwise(pl.col('pseudo_inReplyToUsername'))
      .cast(pl.Int64)
      .alias('pseudo_inReplyToUsername')
)
print(f"✅ pseudo_inReplyToUsername 类型转换完成: String -> Int64")

# 验证类型
print(f"\n📊 关键字段类型验证:")
print(f"  pseudo_author_userName: {df_cleaned['pseudo_author_userName'].dtype}")
print(f"  pseudo_inReplyToUsername: {df_cleaned['pseudo_inReplyToUsername'].dtype}")

✅ 数据加载完成: 508,954 行, 16 列
✅ 布尔列规范化完成: ['isReply', 'author_isBlueVerified']
✅ 时间字段转换完成
✅ pseudo_inReplyToUsername 类型转换完成: String -> Int64

📊 关键字段类型验证:
  pseudo_author_userName: Int64
  pseudo_inReplyToUsername: Int64


## 步骤 3: 添加事件相关时间字段（核心优化）

**关键时间点**:
- 枪击发生: 2025-09-10 (具体时间从数据推断)
- 数据窗口: 事件后72小时内的社交媒体反应

In [4]:
# 分析数据时间范围
print("\n📅 数据时间范围分析:")
min_time = df_cleaned['createdAt'].min()
max_time = df_cleaned['createdAt'].max()
print(f"  最早推文: {min_time}")
print(f"  最晚推文: {max_time}")
print(f"  数据跨度: {(max_time - min_time).total_seconds() / 3600:.1f} 小时")

# 推断枪击发生时间（假设为数据开始前的某个时间点）
# 根据数据最早时间推断：2025-09-11 23:55，枪击应该发生在9月10日
# 保守估计：9月10日下午（美国山地时间），约UTC时间9月10日晚上
SHOOTING_TIMESTAMP = datetime(2025, 9, 10, 20, 0, 0, tzinfo=timezone.utc)  # UTC时间
print(f"\n🎯 枪击事件时间（推断）: {SHOOTING_TIMESTAMP}")

# 添加事件相关字段（修复：Polars datetime 是微秒精度，需要 * 1_000_000）
df_with_event = df_cleaned.with_columns([
    # 距离枪击事件的时间差（小时）
    ((pl.col('createdAt').cast(pl.Int64) - pl.lit(int(SHOOTING_TIMESTAMP.timestamp() * 1_000_000))) / 1_000_000 / 3600).alias('event_time_delta_hours'),
    
    # 事件后时段标签
    pl.when(
        (pl.col('createdAt').cast(pl.Int64) - pl.lit(int(SHOOTING_TIMESTAMP.timestamp() * 1_000_000))) / 1_000_000 / 3600 < 6
    ).then(pl.lit('0-6h'))
    .when(
        (pl.col('createdAt').cast(pl.Int64) - pl.lit(int(SHOOTING_TIMESTAMP.timestamp() * 1_000_000))) / 1_000_000 / 3600 < 12
    ).then(pl.lit('6-12h'))
    .when(
        (pl.col('createdAt').cast(pl.Int64) - pl.lit(int(SHOOTING_TIMESTAMP.timestamp() * 1_000_000))) / 1_000_000 / 3600 < 24
    ).then(pl.lit('12-24h'))
    .when(
        (pl.col('createdAt').cast(pl.Int64) - pl.lit(int(SHOOTING_TIMESTAMP.timestamp() * 1_000_000))) / 1_000_000 / 3600 < 48
    ).then(pl.lit('24-48h'))
    .otherwise(pl.lit('48-72h'))
    .alias('time_window')
])

print(f"\n✅ 事件时间字段添加完成")
print(f"\n时段分布:")
print(df_with_event.group_by('time_window').agg(pl.len().alias('count')).sort('time_window'))


📅 数据时间范围分析:
  最早推文: 2025-09-11 23:55:56+00:00
  最晚推文: 2025-09-13 00:12:32+00:00
  数据跨度: 24.3 小时

🎯 枪击事件时间（推断）: 2025-09-10 20:00:00+00:00

✅ 事件时间字段添加完成

时段分布:
shape: (2, 2)
┌─────────────┬────────┐
│ time_window ┆ count  │
│ ---         ┆ ---    │
│ str         ┆ u32    │
╞═════════════╪════════╡
│ 24-48h      ┆ 415075 │
│ 48-72h      ┆ 93879  │
└─────────────┴────────┘


## 步骤 4: 数据质量检查

In [5]:
# 缺失值检查
key_cols = ['pseudo_id', 'pseudo_author_userName', 'createdAt', 'text']
missing_stats = profiling.missingness_summary(df_with_event, key_cols)
print("📊 缺失值统计 (top 5):")
print(missing_stats.head(5))

# 重复检查
dupes = profiling.duplicate_check(df_with_event, ['pseudo_id'])
print(f"\n🔍 重复推文数: {dupes.height}")

📊 缺失值统计 (top 5):
shape: (5, 4)
┌──────────────────────────┬────────────┬────────────┬────────┐
│ column                   ┆ null_count ┆ null_ratio ┆ is_key │
│ ---                      ┆ ---        ┆ ---        ┆ ---    │
│ str                      ┆ i64        ┆ f64        ┆ bool   │
╞══════════════════════════╪════════════╪════════════╪════════╡
│ pseudo_inReplyToUsername ┆ 259862     ┆ 0.510581   ┆ false  │
│ pseudo_id                ┆ 0          ┆ 0.0        ┆ true   │
│ text                     ┆ 0          ┆ 0.0        ┆ true   │
│ retweetCount             ┆ 0          ┆ 0.0        ┆ false  │
│ replyCount               ┆ 0          ┆ 0.0        ┆ false  │
└──────────────────────────┴────────────┴────────────┴────────┘

🔍 重复推文数: 50


In [6]:
import re

def extract_author_stance_from_bio(bio: str) -> tuple[str, float]:
    """
    从作者bio中提取政治立场预标注
    
    返回: (stance, confidence)
    - stance: 'conservative' | 'liberal' | 'neutral'
    - confidence: 0.0-1.0 (匹配的关键词数量决定)
    """
    if bio is None or not isinstance(bio, str):
        return 'neutral', 0.0
    
    bio_lower = bio.lower()
    
    # 保守派信号词
    conservative_keywords = [
        r'\bmaga\b', r'\btrump\b', r'\bconservative\b', r'\bpatriot\b',
        r'\bamerica first\b', r'\b2a\b', r'\bpro-life\b', r'\bpro life\b',
        r'\bread\w* maga\b', r'\bgod\b.*\bcountry\b', r'\brepublican\b',
        r'\bright\w* wing\b', r'\btea party\b', r'\bliberty\b.*\bfreedom\b',
        r'\b#maga\b', r'\b#trump\b', r'\b#americafirst\b'
    ]
    
    # 自由派信号词
    liberal_keywords = [
        r'\bresist\b', r'\bprogressive\b', r'\bliberal\b', r'\bdemocrat\b',
        r'\bblm\b', r'\bblack lives matter\b', r'\bclimate action\b',
        r'\blgbtq\+?\b', r'\bshe/her\b', r'\bhe/him\b', r'\bthey/them\b',
        r'\bdei\b', r'\bequity\b', r'\binclusion\b', r'\banti[- ]trump\b',
        r'\b#resist\b', r'\b#blm\b', r'\b#metoo\b', r'\bleft\w* activist\b'
    ]
    
    # 计数匹配
    conservative_count = sum(1 for pattern in conservative_keywords if re.search(pattern, bio_lower))
    liberal_count = sum(1 for pattern in liberal_keywords if re.search(pattern, bio_lower))
    
    # 决策逻辑
    if conservative_count > liberal_count and conservative_count > 0:
        return 'conservative', min(conservative_count * 0.2, 1.0)
    elif liberal_count > conservative_count and liberal_count > 0:
        return 'liberal', min(liberal_count * 0.2, 1.0)
    else:
        return 'neutral', 0.0

# 测试函数
test_bios = [
    "MAGA supporter | America First | Pro-life Christian",
    "She/Her | BLM | Resist Trump | Climate Action Now",
    "Software engineer | Coffee lover | Cat dad",
    None
]

print("🧪 测试作者立场提取:")
for bio in test_bios:
    stance, conf = extract_author_stance_from_bio(bio)
    print(f"  Bio: {str(bio)[:50]:<50} → {stance:12} (conf: {conf:.2f})")

🧪 测试作者立场提取:
  Bio: MAGA supporter | America First | Pro-life Christia → conservative (conf: 0.60)
  Bio: She/Her | BLM | Resist Trump | Climate Action Now  → liberal      (conf: 0.80)
  Bio: Software engineer | Coffee lover | Cat dad         → neutral      (conf: 0.00)
  Bio: None                                               → neutral      (conf: 0.00)


## 步骤 5: 作者立场预标注（优化新增）

**目标**: 从作者 bio 中提取政治立场信号，用于后续推文立场分类的辅助判断

**方法**: 基于关键词匹配识别 conservative/liberal 信号

In [7]:
# 加载作者信息
authors_df = io.read_well_known_authors()
print(f"📋 作者元数据: {authors_df.height} 位作者")

# 【优化新增】为作者添加立场预标注
print(f"\n🏷️  正在为作者添加立场预标注...")
stance_results = [
    extract_author_stance_from_bio(bio) 
    for bio in authors_df['author_profile_bio_description'].to_list()
]
stances = [r[0] for r in stance_results]
confidences = [r[1] for r in stance_results]

authors_df = authors_df.with_columns([
    pl.Series('author_stance_prelabel', stances),
    pl.Series('author_stance_confidence', confidences)
])

print(f"✅ 作者立场预标注完成")
print(f"\n立场分布:")
stance_dist = authors_df.group_by('author_stance_prelabel').agg(pl.len().alias('count')).sort('count', descending=True)
print(stance_dist)
print(f"\n有立场信号的作者: {authors_df.filter(pl.col('author_stance_confidence') > 0).height} / {authors_df.height} ({authors_df.filter(pl.col('author_stance_confidence') > 0).height / authors_df.height * 100:.1f}%)")

# 【关键修复】处理 obfuscated_userName
# 去掉 @ 前缀并转换为 Int64
print(f"\n🔍 处理 JOIN KEY:")
print(f"  原始格式: {authors_df['obfuscated_userName'].head(3).to_list()}")

authors_df = authors_df.with_columns(
    pl.col('obfuscated_userName')
      .str.strip_prefix('@')
      .cast(pl.Int64)
      .alias('obfuscated_userName_int')
)

print(f"  转换后: {authors_df['obfuscated_userName_int'].head(3).to_list()}")
print(f"  推文示例: {df_with_event['pseudo_author_userName'].head(3).to_list()}")

# JOIN
df_with_authors = df_with_event.join(
    authors_df,
    left_on='pseudo_author_userName',
    right_on='obfuscated_userName_int',
    how='left'
)

print(f"\n✅ 数据合并完成: {df_with_authors.height:,} 行, {df_with_authors.width} 列")

# 【验证】检查覆盖率
labeled_count = df_with_authors.filter(pl.col('author_stance_prelabel').is_not_null()).height
print(f"\n📊 立场标注覆盖:")
print(f"  有立场标注的推文数: {labeled_count:,} / {df_with_authors.height:,} ({labeled_count / df_with_authors.height * 100:.1f}%)")
if labeled_count > 0:
    with_signal = df_with_authors.filter(pl.col('author_stance_confidence') > 0).height
    print(f"  有立场信号的推文数: {with_signal:,} ({with_signal / df_with_authors.height * 100:.1f}%)")

📋 作者元数据: 4242 位作者

🏷️  正在为作者添加立场预标注...
✅ 作者立场预标注完成

立场分布:
shape: (3, 2)
┌────────────────────────┬───────┐
│ author_stance_prelabel ┆ count │
│ ---                    ┆ ---   │
│ str                    ┆ u32   │
╞════════════════════════╪═══════╡
│ neutral                ┆ 3733  │
│ conservative           ┆ 441   │
│ liberal                ┆ 68    │
└────────────────────────┴───────┘

有立场信号的作者: 509 / 4242 (12.0%)

🔍 处理 JOIN KEY:
  原始格式: ['@292512269517606', '@608765984246811', '@300496301105995']
  转换后: [292512269517606, 608765984246811, 300496301105995]
  推文示例: [683805731746893, 772698505691224, 62423919865749]

✅ 数据合并完成: 508,954 行, 28 列

📊 立场标注覆盖:
  有立场标注的推文数: 44,854 / 508,954 (8.8%)
  有立场信号的推文数: 3,132 (0.6%)


## 步骤 6: 写入 Parquet 缓存

In [8]:
# 写入 parquet 文件供后续分析使用
# 【修复】使用统一的 parquet 目录，而不是各 notebook 自己的目录
output_path = io.PARQUET_DIR / "tweets_enriched.parquet"
print(f"📁 保存路径: {output_path}")
print(f"   (统一目录: {io.PARQUET_DIR})")

io.materialize_parquet(df_with_authors.lazy(), output_path)

print(f"\n✅ Parquet 文件已生成: {output_path}")
print(f"📁 文件大小: {output_path.stat().st_size / 1024 / 1024:.2f} MB")

print(f"\n新增字段:")
print(f"  - event_time_delta_hours: 距枪击事件的小时数")
print(f"  - time_window: 事件后时段标签（0-6h, 6-12h, 12-24h, 24-48h, 48-72h）")
print(f"  - author_stance_prelabel: 作者立场预标注 (conservative/liberal/neutral)")
print(f"  - author_stance_confidence: 立场预标注置信度 (0.0-1.0)")

# 验证新字段
print(f"\n📊 验证保存的字段:")
print(f"  保存前列数: {df_with_authors.width}")
print(f"  应包含字段: event_time_delta_hours, time_window, author_stance_prelabel, author_stance_confidence")

# 读取验证
df_verify = pl.read_parquet(output_path)
print(f"\n✅ 验证读取:")
print(f"  读取后列数: {df_verify.width}")

# 检查关键字段
required_fields = ['author_stance_prelabel', 'author_stance_confidence', 'event_time_delta_hours', 'time_window']
missing_fields = [f for f in required_fields if f not in df_verify.columns]
if missing_fields:
    print(f"\n❌ 警告: 以下字段缺失 - {missing_fields}")
else:
    print(f"\n✅ 所有必需字段都已保存")
    # 显示作者立场字段示例
    print(f"\n📊 作者立场字段示例:")
    print(df_verify.select(['pseudo_author_userName', 'author_stance_prelabel', 'author_stance_confidence', 'text']).head(5))

📁 保存路径: /workspace/src/notebooks/parquet/tweets_enriched.parquet
   (统一目录: /workspace/src/notebooks/parquet)

✅ Parquet 文件已生成: /workspace/src/notebooks/parquet/tweets_enriched.parquet
📁 文件大小: 59.05 MB

新增字段:
  - event_time_delta_hours: 距枪击事件的小时数
  - time_window: 事件后时段标签（0-6h, 6-12h, 12-24h, 24-48h, 48-72h）
  - author_stance_prelabel: 作者立场预标注 (conservative/liberal/neutral)
  - author_stance_confidence: 立场预标注置信度 (0.0-1.0)

📊 验证保存的字段:
  保存前列数: 28
  应包含字段: event_time_delta_hours, time_window, author_stance_prelabel, author_stance_confidence

✅ 验证读取:
  读取后列数: 28

✅ 所有必需字段都已保存

📊 作者立场字段示例:
shape: (5, 4)
┌────────────────────────┬────────────────────────┬────────────────────────┬───────────────────────┐
│ pseudo_author_userName ┆ author_stance_prelabel ┆ author_stance_confiden ┆ text                  │
│ ---                    ┆ ---                    ┆ ce                     ┆ ---                   │
│ i64                    ┆ str                    ┆ ---                    ┆ str            

## ✅ 数据接入完成！

**新增核心字段**:
- `event_time_delta_hours`: 距枪击事件的时间差（小时）
- `time_window`: 事件后时段标签
- **【优化新增】** `author_stance_prelabel`: 作者立场预标注 (conservative/liberal/neutral)
- **【优化新增】** `author_stance_confidence`: 立场预标注置信度 (0.0-1.0)

**下一步**: 运行 `01_content_semantics.ipynb` 进行情感与叙事分析（将使用author立场辅助分类）